# Combine Sports Reference and Kenpom data into one dataframe

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

# head_names = ['School', 'W-L%', 'SRS', 'SOS', 'TmPts', 'OppPts', 
#               'Pace', 'ORtg', 'FTr', '3PAr', 'TS%', 'TRB%', 'AST%', 
#               'STL%', 'BLK%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA']

kpHeadNames=['Rank','School','Conf','WL','AdjEM','AdjO','AdjD','AdjT','Luck','AdjEM','OppO','OppD','NCSOS_AdjEM','junk']

def num(s):
    try:
        return int(s)
    except ValueError:
        return float(s)


years = ['2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010']


In [11]:
kpData=pd.read_csv('data/kp/kp-2017.csv',names=kpHeadNames).drop('junk',axis=1)
teamNamesKP=kpData.School.values

newTeamList=[]
for t in teamNamesKP:
    newTeamList.append(t.lower().replace(" ","-").replace("st.","state"))
teamNamesKP=newTeamList

In [12]:
srData=pd.read_csv('data/team/team_stats_2017.csv')
teamNamesBR=srData.School

Matchup all names for consistency

In [13]:
print(len(teamNamesBR))
print(len(teamNamesKP))

351
351


In [14]:
teamNameMismatch=[]
for teamName in teamNamesKP:
    if teamName not in set(list(teamNamesBR)):
        teamNameMismatch.append(teamName)
len(teamNameMismatch)

45

In [15]:
teamNamesKP

['gonzaga',
 'villanova',
 'north-carolina',
 'kentucky',
 'florida',
 'kansas',
 'west-virginia',
 'wichita-state',
 'louisville',
 'oregon',
 'smu',
 'virginia',
 'baylor',
 'duke',
 "saint-mary's",
 'ucla',
 'iowa-state',
 'arizona',
 'purdue',
 'michigan',
 'wisconsin',
 'oklahoma-state',
 'cincinnati',
 'south-carolina',
 'butler',
 'florida-state',
 'notre-dame',
 'creighton',
 'tcu',
 'kansas-state',
 'xavier',
 'marquette',
 'vanderbilt',
 'rhode-island',
 'arkansas',
 'wake-forest',
 'minnesota',
 'northwestern',
 'dayton',
 'michigan-state',
 'texas-tech',
 'miami-fl',
 'middle-tennessee',
 'indiana',
 'clemson',
 'maryland',
 'utah',
 'vcu',
 'illinois-state',
 'virginia-tech',
 'seton-hall',
 'houston',
 'usc',
 'nevada',
 'syracuse',
 'alabama',
 'tennessee',
 'princeton',
 'unc-wilmington',
 'providence',
 'georgia',
 'california',
 'vermont',
 'texas-a&m',
 'oklahoma',
 'illinois',
 'east-tennessee-state',
 'ucf',
 'georgetown',
 'texas',
 'iowa',
 'colorado',
 'ohio-sta

In [16]:
teamNameConversionKP2BR={
'smu': 'southern-methodist',
 "saint-mary's": "saint-mary's-(ca)",
 'tcu': 'texas-christian',
 'miami-fl': 'miami-(fl)',
 'vcu': 'virginia-commonwealth',
 'usc': 'southern-california',
 'unc-wilmington': 'north-carolina-wilmington',
 'california': 'university-of-california',
 'ucf': 'central-florida',
 'ut-arlington': 'texas-arlington',
 'byu': 'brigham-young',
 'state-bonaventure': 'st.-bonaventure',
 'loyola-chicago': 'loyola-(il)',
 "state-john's": "st.-john's-(ny)",
 'unc-asheville': 'north-carolina-asheville',
 'unc-greensboro': 'north-carolina-greensboro',
 'albany': 'albany-(ny)',
 'fort-wayne': 'ipfw',
 'penn': 'pennsylvania',
 'lsu': 'louisiana-state',
 'texas-a&m-corpus-chris': 'texas-a&m-corpus-christi',
 'uab': 'alabama-birmingham',
 'utep': 'texas-el-paso',
 'umbc': 'maryland-baltimore-county',
 "mount-state-mary's": "mount-st.-mary's",
 'umkc': 'missouri-kansas-city',
 'loyola-md': 'loyola-(md)',
 'usc-upstate': 'south-carolina-upstate',
 'little-rock': 'arkansas-little-rock',
 'unlv': 'nevada-las-vegas',
 'bowling-green': 'bowling-green-state',
 'utsa': 'texas-san-antonio',
 'liu-brooklyn': 'long-island-university',
 'state-francis-pa': 'saint-francis-(pa)',
 'fiu': 'florida-international',
 'miami-oh': 'miami-(oh)',
 'the-citadel': 'citadel',
 'umass-lowell': 'massachusetts-lowell',
 'ut-rio-grande-valley': 'texas-rio-grande-valley',
 'detroit': 'detroit-mercy',
 'prairie-view-a&m': 'prairie-view',
 'southern-miss': 'southern-mississippi',
 'grambling-state': 'grambling',
 'central-connecticut': 'central-connecticut-state',
 'state-francis-ny': 'st.-francis-(ny)'
}

In [17]:
teamNamesKPConverted=[]
for teamName in teamNamesKP:
    if teamName in teamNameConversionKP2BR.keys():
        teamNamesKPConverted.append(teamNameConversionKP2BR[teamName])
    else:
        teamNamesKPConverted.append(teamName)
len(teamNamesKPConverted)

351

Remaining number of mismatches

In [18]:
teamNameMismatch=[]
for teamName in teamNamesKPConverted:
    if teamName not in set(list(teamNamesBR)):
        teamNameMismatch.append(teamName)
len(teamNameMismatch)

0

In [19]:

teamNameMismatch

[]

In [20]:
tquery='california-davis'
for teamName in teamNamesBR:
    if tquery in teamName:
        print(tquery, ' is in')
        

# Combine dataframes

In [21]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt


In [22]:
kpData.School.values

array(['Gonzaga', 'Villanova', 'North Carolina', 'Kentucky', 'Florida',
       'Kansas', 'West Virginia', 'Wichita St.', 'Louisville', 'Oregon',
       'SMU', 'Virginia', 'Baylor', 'Duke', "Saint Mary's", 'UCLA',
       'Iowa St.', 'Arizona', 'Purdue', 'Michigan', 'Wisconsin',
       'Oklahoma St.', 'Cincinnati', 'South Carolina', 'Butler',
       'Florida St.', 'Notre Dame', 'Creighton', 'TCU', 'Kansas St.',
       'Xavier', 'Marquette', 'Vanderbilt', 'Rhode Island', 'Arkansas',
       'Wake Forest', 'Minnesota', 'Northwestern', 'Dayton',
       'Michigan St.', 'Texas Tech', 'Miami FL', 'Middle Tennessee',
       'Indiana', 'Clemson', 'Maryland', 'Utah', 'VCU', 'Illinois St.',
       'Virginia Tech', 'Seton Hall', 'Houston', 'USC', 'Nevada',
       'Syracuse', 'Alabama', 'Tennessee', 'Princeton', 'UNC Wilmington',
       'Providence', 'Georgia', 'California', 'Vermont', 'Texas A&M',
       'Oklahoma', 'Illinois', 'East Tennessee St.', 'UCF', 'Georgetown',
       'Texas', 'Iowa', 'Colo

In [23]:
for yr in np.arange(2017,2001,-1):
    # pull kenpom data
    print(yr)
    kpData=pd.read_csv('data/kp/kp-'+str(yr)+'.csv',names=kpHeadNames).drop('junk',axis=1)
    teamNamesKP=kpData.School.values

    #basic formatting of school names
    newTeamList=[]
    for t in teamNamesKP:
        newTeamList.append(t.lower().replace(" ","-").replace("st.","state"))
    teamNamesKP=newTeamList

    # convert remaining names
    teamNamesKPConverted=[]
    for teamName in teamNamesKP:
        if teamName in teamNameConversionKP2BR.keys():
            teamNamesKPConverted.append(teamNameConversionKP2BR[teamName])
        else:
            teamNamesKPConverted.append(teamName)

    kpData.School=teamNamesKPConverted
    
    

    srData=pd.read_csv('data/team/team_stats_'+str(yr)+'.csv')
    teamNamesBR=srData.School

    allData=srData.merge(kpData,how='inner',on='School')
    allData=allData.rename(columns={'W%': 'wpct'})
    
    allData.to_csv("data/combined_team_stats_" + str(yr) + ".csv")
    
#     break
    # allData.sort_values(by='W-L%',ascending=False)
#     wpct=[]
#     for r in allData.itertuples():
#         w,l = r.WL.split('-')
#         wpct.append(float(w)/(float(w)+float(l)))
#     allData['wpct2']=wpct
#     #     print()
#     plt.figure()
#     plt.title(yr)
#     allData[['wpct','wpct2']]
#     plt.hist((allData['wpct']-allData['wpct2']));

2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004
2003
2002


In [11]:
allData

,School,wpct,SRS,SOS,TmPts,OppPts,Pace,ORtg,FTr,3PAr,...,WL,AdjEM,AdjO,AdjD,AdjT,Luck,AdjEM.1,OppO,OppD,NCSOS_AdjEM
0,air-force,0.321,-0.95,3.23,1605,1722,NaN,NaN,0.288,0.485,...,9-19,-1.64,103.8,105.5,54.2,-0.015,4.30,105.9,101.6,-0.75
1,akron,0.323,-4.84,0.36,2154,2304,NaN,NaN,0.350,0.311,...,9-21,-5.94,105.2,111.2,65.6,-0.026,1.27,105.4,104.2,-1.37
2,alabama-a&m,0.655,-11.33,-15.81,2318,2123,NaN,NaN,0.238,0.339,...,17-10,-9.53,92.2,101.7,79.0,0.002,-18.64,92.8,111.5,-15.03
3,alabama-birmingham,0.433,5.56,2.96,2110,2032,NaN,NaN,0.243,0.332,...,13-17,6.18,102.5,96.4,67.7,-0.143,2.74,104.7,102.0,-2.90
4,alabama-state,0.594,-13.25,-14.15,2095,2067,NaN,NaN,0.287,0.317,...,18-12,-12.62,90.1,102.7,63.1,0.081,-14.95,95.8,110.8,-5.12
5,alabama,0.771,16.93,7.67,2658,2334,NaN,NaN,0.300,0.339,...,27-8,18.73,114.0,95.3,68.0,0.076,8.59,107.4,98.8,-1.56
6,albany-(ny),0.286,-16.99,-8.85,1593,1821,NaN,NaN,0.157,0.323,...,8-20,-20.29,87.0,107.3,62.7,0.039,-9.58,98.2,107.8,-4.05
7,alcorn-state,0.677,-11.28,-14.18,2473,2374,NaN,NaN,0.258,0.237,...,20-10,-8.71,100.1,108.8,73.0,0.097,-14.53,95.3,109.8,2.60
8,american,0.600,-5.00,-6.20,1977,1941,NaN,NaN,0.267,0.401,...,18-12,-4.01,101.0,105.1,62.9,0.057,-7.70,98.5,106.2,-3.16
9,appalachian-state,0.357,-7.53,-0.65,2140,2211,NaN,NaN,0.282,0.283,...,8-18,-6.25,102.1,108.3,72.6,-0.006,0.93,103.3,102.4,2.60
